**DSCI100 Final Project Report**

Group 19 (Aneeta, Avishka, Hannah, Ksenia)

**Introduction**

In this project, analysis on data from the High Time Resolution Universe Pulsar Survey (Lyon, 2016) will be done to gain a better understanding of how to better classify Neutron stars (also known as pulsars).

Studying pulsars allows researchers to explore “space-time, the interstellar medium, and states of matter.” However, radio signals received to identify pulsars are obstructed with radio frequency interference (RFI) and noise. As a result, there is demand for advanced learning tools that will be able to automatically label pulsar candidates.

If the pulse’s signal is integrated with respect to its rotational period, a unique integrated profile can be created for the pulse. A DM-SNR curve plots the spectral supernova remnants (SNR) as a function of the trail dispersion measure (DM).

The question that this project will answer is “Which variable, between integrated profile and DM-SNR, is a better predictor for class?”

The dataset that is used for this project provides information on 17,898 candidates—16,259 are examples of RFI/noise, and 1,639 are examples of real pulsars. 8 variables corresponding to data about the integrated profile and DM-SNR curve of the candidates, will be further analyzed in the project to answer the research question.


**Methods & Results**

**References**

Lyon, R., Stappers, B., Cooper, S., Brooke, J. M., & Knowles, J. D. (2016). Fifty Years of Pulsar Candidate Selection: From simple filters to a new principled real-time classification approach. Monthly Notices of the Royal Astronomical Society, 459(1), 1104-1123. https://doi.org/10.1093/mnras/stw656

